## Day 30 Lecture 2 Assignment

In this assignment, we will learn about random forests. We will use the google play store dataset loaded below.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/googleplaystore.csv')

df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [3]:
for column in df.columns:
  print(f'Counts for {column}:\n', df[column].value_counts())

Counts for App:
 ROBLOX                                               9
CBS Sports App - Scores, News, Stats & Watch Live    8
Duolingo: Learn Languages Free                       7
Candy Crush Saga                                     7
ESPN                                                 7
                                                    ..
AzadBazar.af                                         1
BIGO LIVE - Live Stream                              1
Gangster Town                                        1
Planning Center Services                             1
Egg                                                  1
Name: App, Length: 9660, dtype: int64
Counts for Category:
 FAMILY                 1972
GAME                   1144
TOOLS                   843
MEDICAL                 463
BUSINESS                460
PRODUCTIVITY            424
PERSONALIZATION         392
COMMUNICATION           387
SPORTS                  384
LIFESTYLE               382
FINANCE                 366
HEALTH_AND

In this assignment, you will work more independently. Perform the following steps:
    
1. Select which columns are best suited to predict whether the rating is above 4.5
2. Process the data (including transforming to the correct column type, removing missing values, creating dummy variables, and removing irrelevant variables)
3. Create a random forest model and evaluate
4. Using grid search cross validation, tweak the parameters to produce a better performing model
5. Show and discuss your results

Good luck!

In [4]:
df['Best_Rating'] = df['Rating'].map(lambda x: 1 if x > 4.5 else 0)

In [5]:
df2 = df.drop(columns = ['Genres', 'Category', 'App', 'Rating', 'Size', 'Last Updated', 'Current Ver', 'Android Ver'])
df2.head(2)

,Reviews,Installs,Type,Price,Content Rating,Best_Rating
0,159,"10,000+",Free,0,Everyone,0
1,967,"500,000+",Free,0,Everyone,0


In [6]:
df2.isnull().sum()

Reviews           0
Installs          0
Type              1
Price             0
Content Rating    1
Best_Rating       0
dtype: int64

In [7]:
df2 = df2.dropna()

In [8]:
df2.isnull().sum()

Reviews           0
Installs          0
Type              0
Price             0
Content Rating    0
Best_Rating       0
dtype: int64

In [9]:
df2.dtypes

Reviews           object
Installs          object
Type              object
Price             object
Content Rating    object
Best_Rating        int64
dtype: object

In [10]:
y = df2['Best_Rating']
x = df2.drop(columns='Best_Rating')

In [11]:
x.nunique()

Reviews           6001
Installs            20
Type                 2
Price               92
Content Rating       6
dtype: int64

In [12]:
for column in x.columns:
  print(f'Counts for {column}:\n', x[column].value_counts())

Counts for Reviews:
 0          595
1          272
2          214
3          175
4          137
          ... 
9856         1
3840         1
410303       1
1003269      1
776730       1
Name: Reviews, Length: 6001, dtype: int64
Counts for Installs:
 1,000,000+        1579
10,000,000+       1252
100,000+          1169
10,000+           1054
1,000+             907
5,000,000+         752
100+               719
500,000+           539
50,000+            479
5,000+             477
100,000,000+       409
10+                386
500+               330
50,000,000+        289
50+                205
5+                  82
500,000,000+        72
1+                  67
1,000,000,000+      58
0+                  14
Name: Installs, dtype: int64
Counts for Type:
 Free    10039
Paid      800
Name: Type, dtype: int64
Counts for Price:
 0         10039
$0.99       148
$2.99       129
$1.99        73
$4.99        72
          ...  
$15.46        1
$25.99        1
$89.99        1
$30.99        1
$1.50      

In [13]:
# to change installs to a number I have to take away the + sign and the comma
x['Installs'] = x['Installs'].str.strip('+')

In [14]:
x['Installs'] = x['Installs'].str.replace(',', '')

In [15]:
x['Installs'] = x['Installs'].astype(int)

In [16]:
x.dtypes

Reviews           object
Installs           int64
Type              object
Price             object
Content Rating    object
dtype: object

In [17]:
x['Installs'].value_counts()

1000000       1579
10000000      1252
100000        1169
10000         1054
1000           907
5000000        752
100            719
500000         539
50000          479
5000           477
100000000      409
10             386
500            330
50000000       289
50             205
5               82
500000000       72
1               67
1000000000      58
0               14
Name: Installs, dtype: int64

In [18]:
# changing Installs to high and low so I can one-hot-encode
x['Installs'] = x['Installs'].map(lambda x: 'High' if x > 1000000 else 'Low' )
x.head()

,Reviews,Installs,Type,Price,Content Rating
0,159,Low,Free,0,Everyone
1,967,Low,Free,0,Everyone
2,87510,High,Free,0,Everyone
3,215644,High,Free,0,Teen
4,967,Low,Free,0,Everyone


In [19]:
x.Price = x.Price.replace('[\$,]', '', regex=True).astype(float)

In [20]:
x.dtypes

Reviews            object
Installs           object
Type               object
Price             float64
Content Rating     object
dtype: object

In [21]:
x['Reviews'] = x['Reviews'].astype(int)

In [22]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10839 entries, 0 to 10840
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Reviews         10839 non-null  int64  
 1   Installs        10839 non-null  object 
 2   Type            10839 non-null  object 
 3   Price           10839 non-null  float64
 4   Content Rating  10839 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 508.1+ KB


In [23]:
pd.get_dummies(x, drop_first=True)

,Reviews,Price,Installs_Low,Type_Paid,Content Rating_Everyone,Content Rating_Everyone 10+,Content Rating_Mature 17+,Content Rating_Teen,Content Rating_Unrated
0,159,0.0,1,0,1,0,0,0,0
1,967,0.0,1,0,1,0,0,0,0
2,87510,0.0,0,0,1,0,0,0,0
3,215644,0.0,0,0,0,0,0,1,0
4,967,0.0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
10836,38,0.0,1,0,1,0,0,0,0
10837,4,0.0,1,0,1,0,0,0,0
10838,3,0.0,1,0,1,0,0,0,0
10839,114,0.0,1,0,0,0,1,0,0


In [24]:
x = pd.concat([x, pd.get_dummies(x['Installs'], drop_first=True)], axis=1)
x = pd.concat([x, pd.get_dummies(x['Type'], drop_first=True)], axis=1)
x = pd.concat([x, pd.get_dummies(x['Content Rating'], drop_first=True)], axis=1)
x = x.drop(columns=['Installs', 'Type', 'Content Rating'])
x

,Reviews,Price,Low,Paid,Everyone,Everyone 10+,Mature 17+,Teen,Unrated
0,159,0.0,1,0,1,0,0,0,0
1,967,0.0,1,0,1,0,0,0,0
2,87510,0.0,0,0,1,0,0,0,0
3,215644,0.0,0,0,0,0,0,1,0
4,967,0.0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
10836,38,0.0,1,0,1,0,0,0,0
10837,4,0.0,1,0,1,0,0,0,0
10838,3,0.0,1,0,1,0,0,0,0
10839,114,0.0,1,0,0,0,1,0,0


In [25]:
y

0        0
1        0
2        1
3        0
4        0
        ..
10836    0
10837    1
10838    0
10839    0
10840    0
Name: Best_Rating, Length: 10839, dtype: int64

Create a random forest model and evaluate

Using grid search cross validation, tweak the parameters to produce a better performing model

Show and discuss your results

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import train_test_split

In [27]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [28]:
rfc = RandomForestClassifier()
rfc.fit(x_train, y_train)
rfc.feature_importances_

array([9.41998252e-01, 4.01047658e-02, 4.34071653e-03, 4.58618185e-03,
       2.62268438e-03, 1.88136910e-03, 1.97461907e-03, 2.46819098e-03,
       2.32201142e-05])

In [29]:
best_features = pd.DataFrame()
best_features['Names'] = x.columns
best_features['Scores'] = rfc.feature_importances_

In [30]:
best_features

,Names,Scores
0,Reviews,0.941998
1,Price,0.040105
2,Low,0.004341
3,Paid,0.004586
4,Everyone,0.002623
5,Everyone 10+,0.001881
6,Mature 17+,0.001975
7,Teen,0.002468
8,Unrated,0.000023


In [31]:
x.shape[0]

10839

In [32]:
grid_param = {
    'n_estimators': [100, 300, 500, 800, 1000],
    'criterion': ['gini', 'entropy'],
    'bootstrap': [True, False]
}

In [33]:
gd_sr = GridSearchCV(estimator=rfc,
                     param_grid=grid_param,
                     scoring='accuracy',
                     cv=5,
                     n_jobs=-1)

In [34]:
gd_sr.fit(x_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [35]:
best_parameters = gd_sr.best_params_
print(best_parameters)

{'bootstrap': False, 'criterion': 'entropy', 'n_estimators': 100}


In [36]:
best_result = gd_sr.best_score_
print(best_result)

0.7937950267409896
